In [1]:
pip install yfinance

In [5]:
pip install seaborn

In [6]:
pip install matplotlib

In [7]:
pip install pandas

In [8]:
pip install cufflinks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.2 MB/s eta 0:00:00


In [9]:
pip install plotly

In [10]:
pip install statsmodels

In [11]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web

In [12]:
RISKY_ASSET = 'BBRI.JK'
START_DATE = '2018-01-01'
END_DATE = '2023-12-31'

In [13]:
# three factors
df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
                                start=START_DATE)[0]
df_three_factor.index = df_three_factor.index.format()

# momentum factor
df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench',
                       start=START_DATE)[0]
df_mom.index = df_mom.index.format()

# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()

<ipython-input-13-455cdfe64dd0>:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
<ipython-input-13-455cdfe64dd0>:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
<ipython-input-13-455cdfe64dd0>:4: FutureWarning: PeriodIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  df_three_factor.index = df_three_factor.index.format()
<ipython-input-13-455cdfe64dd0>:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version

In [15]:
asset_df = yf.download(RISKY_ASSET,
                      start=START_DATE,
                      end=END_DATE,
                      # adjusted=True,  # Removed 'adjusted' argument
                      progress=False)

# If you need adjusted closing prices, you can adjust them manually:
asset_df['Adj Close'] = asset_df['Close'] * asset_df['Adj Close'] / asset_df['Close']

print(f'Downloaded {asset_df.shape[0]} rows of data.')

Downloaded 1493 rows of data.


In [17]:
y = asset_df['Adj Close'].resample('M') \
                        .last() \
                        .pct_change() \
                        .dropna()

y.index = y.index.strftime('%Y-%m')
y.name = 'return'

<ipython-input-17-4b6e92249904>:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  y = asset_df['Adj Close'].resample('M') \


##Merge the datasets for the four-factor models

In [18]:
# join all datasets on the index
four_factor_data = df_three_factor.join(df_mom).join(y)

# rename columns
four_factor_data.columns = ['mkt', 'smb', 'hml', 'rf', 'mom', 'rtn']

# divide everything (except returns) by 100
four_factor_data.loc[:, four_factor_data.columns != 'rtn'] /= 100

# select period of interest
four_factor_data = four_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
four_factor_data['excess_rtn'] = four_factor_data.rtn - four_factor_data.rf

four_factor_data.head()

,mkt,smb,hml,rf,mom,rtn,excess_rtn
2018-02,-0.0365,0.0026,-0.0104,0.0011,0.0355,0.021621,0.020521
2018-03,-0.0235,0.0406,-0.0020,0.0011,-0.0113,-0.047619,-0.048719
2018-04,0.0029,0.0113,0.0054,0.0014,0.0037,-0.078223,-0.079623
2018-05,0.0265,0.0526,-0.0322,0.0014,0.0387,-0.043478,-0.044878
2018-06,0.0048,0.0114,-0.0233,0.0014,-0.0236,-0.077922,-0.079322


##Merge the datasets for the five-factor models

In [19]:
# join all datasets on the index
five_factor_data = df_five_factor.join(y)

# rename columns
five_factor_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma', 'rf', 'rtn']

# divide everything (except returns) by 100
five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100

# select period of interest
five_factor_data = five_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf

five_factor_data.head()

,mkt,smb,hml,rmw,cma,rf,rtn,excess_rtn
2018-02,-0.0365,0.0032,-0.0104,0.0048,-0.0228,0.0011,0.021621,0.020521
2018-03,-0.0235,0.0360,-0.0020,-0.0042,-0.0002,0.0011,-0.047619,-0.048719
2018-04,0.0029,0.0093,0.0054,-0.0244,0.0128,0.0014,-0.078223,-0.079623
2018-05,0.0265,0.0475,-0.0322,-0.0205,-0.0153,0.0014,-0.043478,-0.044878
2018-06,0.0048,0.0080,-0.0233,0.0080,0.0020,0.0014,-0.077922,-0.079322


##Estimate the four-factor model:

In [20]:
four_factor_model = smf.ols(formula='excess_rtn ~ mkt + smb + hml + mom',
                           data=four_factor_data).fit()

print(four_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     5.428
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           0.000769
Time:                        10:02:14   Log-Likelihood:                 94.000
No. Observations:                  71   AIC:                            -178.0
Df Residuals:                      66   BIC:                            -166.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0085      0.008      1.058      0.2

##Estimate the five-factor model:

In [21]:
five_factor_model = smf.ols(
formula='excess_rtn ~ mkt + smb + hml + rmw + cma',
data=five_factor_data
).fit()

print(five_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     5.187
Date:                Thu, 28 Nov 2024   Prob (F-statistic):           0.000457
Time:                        10:02:55   Log-Likelihood:                 95.822
No. Observations:                  71   AIC:                            -179.6
Df Residuals:                      65   BIC:                            -166.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0124      0.008      1.527      0.1